In [1]:
import re
import pprint as pp
import numpy as np
import pdb
import pandas as pd
import statsmodels.regression as smreg
import statsmodels.api as sm
import statsmodels.formula.api as smf
from patsy import dmatrices, dmatrix, demo_data

/usr/local/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


# Fun with numpy.dtype

In [2]:
# scalar and subarray dtypes 
one_x_one_shape_dt = np.dtype([('my_sing_type', 'f8', (1,1))]) # 2-D subarray 1 row, 1 col
singleton_shape_dt = np.dtype([('my_sing_type', 'f8', (1,))])  # 1-D subarray 1 row
no_shape_dt = np.dtype([('my_type', 'f8')]) # 0-D i.e., no subarray, i.e., scalar

arry_a = np.ndarray(shape=(3,), dtype = one_x_one_shape_dt)
arry_b = np.ndarray(shape=(3,), dtype = singleton_shape_dt)
arry_c = np.ndarray(shape=(3,), dtype = no_shape_dt)

print(arry_a.shape) # (3,)
print(arry_b.shape) # (3,)
print(arry_c.shape) # (3)

print(arry_a[0].shape) 
print(arry_b[0].shape)
print(arry_c[0].shape)

print(arry_a[0][0].shape)
print(arry_b[0][0].shape)
print(arry_c[0][0].shape)

print(arry_a[0][0].__class__) # shape (1,1) gives ndarray
print(arry_b[0][0].__class__) # shape (1,) gives ndarray
print(arry_c[0][0].__class__) # shape () gives float


(3,)
(3,)
(3,)
()
()
()
(1, 1)
(1,)
()
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.float64'>


In [7]:
dt_names = ['test1', 'test2']
dt_formats = [ ('f8', (3,)), ('U8', (3,2))]

# dict pattern
test_type = np.dtype({
        'names': dt_names,
        'formats': dt_formats
})

n = 5
mydat = np.empty(shape=(n,), dtype=test_type)
for i in range(n):
    mydat['test1'][i].fill(i+1)
    for ij, al in enumerate(['A','B','C']):
        mydat['test2'][i][ij] = al + '_' + str(i+1) + '_' + str(ij+1)

print('test1')
print(mydat['test1'])

print('test2')
print(mydat['test2'])

# looks like we can slice the middle row *across* different shape subarrays. cool.
print('2nd row slice through both arrays')
pp.pprint(mydat[...][1]) 

test1
[[ 1.  1.  1.]
 [ 2.  2.  2.]
 [ 3.  3.  3.]
 [ 4.  4.  4.]
 [ 5.  5.  5.]]
test2
[[['A_1_1' 'A_1_1']
  ['B_1_2' 'B_1_2']
  ['C_1_3' 'C_1_3']]

 [['A_2_1' 'A_2_1']
  ['B_2_2' 'B_2_2']
  ['C_2_3' 'C_2_3']]

 [['A_3_1' 'A_3_1']
  ['B_3_2' 'B_3_2']
  ['C_3_3' 'C_3_3']]

 [['A_4_1' 'A_4_1']
  ['B_4_2' 'B_4_2']
  ['C_4_3' 'C_4_3']]

 [['A_5_1' 'A_5_1']
  ['B_5_2' 'B_5_2']
  ['C_5_3' 'C_5_3']]]
2nd row slice through both arrays
([2.0, 2.0, 2.0], [['A_2_1', 'A_2_1'], ['B_2_2', 'B_2_2'], ['C_2_3', 'C_2_3']])


# Back to fit bucket dtypes

In [4]:
# demo data from patsy
data = demo_data('y', 'a', 'x0','x1')

# patsy can tell us about the rhs design matrix
lhs = 'y'
rhs = 'x0 + x1 + a'
formula = lhs + '~' + rhs

# to scrape the fit we want the names and number of the rhs terms
rhs_dmat = dmatrix(rhs, data=data)
rhs_di = rhs_dmat.design_info
print(rhs_di.column_names)
print(rhs_di.column_name_indexes)

['Intercept', 'a[T.a2]', 'x0', 'x1']
OrderedDict([('Intercept', 0), ('a[T.a2]', 1), ('x0', 2), ('x1', 3)])


In [5]:
# make the ci interval a dtype of its own
# and build it into the fit_coef_dt
ci_dt = np.dtype([('lower', '<f8'), ('upper', '<f8')])

# define the data type
fit_coef_dt = np.dtype([('idx','<u8'),# rhs column index
                         ('coef', '<f8',), # parameter estimate
                         ('se','<f8',), # SE
                         ('ci',ci_dt)]) # conf interval



In [6]:
# Now consult the the model rhs design matrix and build the fit part of the
# fit bucket dtype.

# Option 1 is an 1-D ndarray shape=(1,) of a single compound data type
# wrapped around all the parameter info. This allows key-value access
# down into each parameter by name but no slicing across parameters


In [7]:
# Option 1: make one big master dtype for this rhs design matrix
# and make the fit object a singleton array of this dtype

# one big compound data type ... a different
# dtype for different design matrices.
fit_dt = np.dtype([(name,fit_coef_dt) \
                       for name in rhs_di.column_names])

# the fit is an ndarray of 1 of these dtypes
fit = np.empty(shape=(1,), dtype=fit_dt)
print(fit.__class__)
print(fit.dtype.names)

# Advantages: access to parameter info by column name
print(fit['Intercept']['coef'])
print(fit['x0']['ci'])

# Disadvantages: cannot slice *across* coefs to extract a named field.
try:
    fit[...]['coef']
except Exception as fail:
    print(fail.args)
    


<class 'numpy.ndarray'>
('Intercept', 'a[T.a2]', 'x0', 'x1')
[  2.34142453e-316]
[(6.9155226262719e-310, 6.9155319292632e-310)]
('no field of name coef',)


In [8]:
# Toy example: building up an (empty) fit grid with

# one of these per channel and sample in the fit grid
fit_bucket_dt = np.dtype([('fit', fit_dt),
                          ('diag', 'O')]) # stub for diag

n_chans  = 4
n_points = 10
fit_grid = np.ndarray(shape=(n_chans, n_points), 
                      dtype=fit_bucket_dt)

def fill_fit_bucket(fit_bucket, fit):
    for i,f in enumerate(fit_bucket['fit'].dtype.names):
        # TO DO: scrape the values from fit into the fit_bucket here
        pass

this_fit = None
for ch in [0]: #range(n_chans):
    for pt in [0]: #range(n_points):
        y_dat = data['y']
        this_model = smreg.linear_model.OLS(data['y'], rhs_dmat)
        this_fit = this_model.fit()
        fill_fit_bucket(fit_grid[0,0],this_fit)


# now we **can** slice into the fit_grid[chan_idx,timepoint_idx] and
# access parameter information by name ...
# 
# 
# Example: rERPs = ['coef'] field over time
fit_grid[0,:]['fit']['Intercept']['coef']

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

# SCRATCH WORK BELOW HERE ...

In [9]:
# So far what gets scraped from the model fit object is hardcoded.

# A more mature version would take a list of fit object attributes
# and return a compound data type with the right number and type and shape 
# of slots, e.g., 
# 
# def get_coef_dt(model_fit_object, list_of_fit_values):
#    ...
#   return np.dtype(for_these_fit_values)

In [10]:
# But. There are a bunch of different kinds of attributes in the fit object that
# have to be handled case by case ... see below.

# These are all the class names currently 
# 'DataFrame' 'OLS' 'Series' 'bool' 'dict' 'float' 'float64' 'int' 'method'
# 'ndarray' 'str']

fit = smf.ols(formula, data=data).fit()
#print(dir(fit))

attrs = [a for a in dir(fit) if re.match('^_.+', a) is None]
fit_classes = []
for a in attrs:
    aval = getattr(fit, a)   
    print(a, aval.__class__.__name__)
    fit_classes.append(aval.__class__.__name__)

fit_classes = np.unique(fit_classes)
print(fit_classes) # These are the cases to deal with



HC0_se Series
HC1_se Series
HC2_se Series
HC3_se Series
aic float64
bic float64
bse Series
centered_tss float64
compare_f_test method
compare_lm_test method
compare_lr_test method
condition_number float64
conf_int method
conf_int_el method
cov_HC0 ndarray
cov_HC1 ndarray
cov_HC2 ndarray
cov_HC3 ndarray
cov_kwds dict
cov_params method
cov_type str
df_model float
df_resid float64
eigenvals ndarray
el_test method
ess float64
f_pvalue float64
f_test method
fittedvalues Series
fvalue float64
get_influence method
get_prediction method
get_robustcov_results method
initialize method
k_constant int
llf float64
load method
model OLS
mse_model float64
mse_resid float64
mse_total float64
nobs float
normalized_cov_params DataFrame
outlier_test method
params Series
predict method
pvalues Series
remove_data method
resid Series
resid_pearson ndarray
rsquared float64
rsquared_adj float64
save method
scale float64
ssr float64
summary method
summary2 method
t_test method
tvalues Series
uncentered_tss flo

In [11]:
# Suppose we wanted to build a data type to hold these values.
fetch_these = ['coefs', 'bse', 'conf_int', 'aic']

# We need to know name, dtype, and shape of each one
# 
for i,f in enumerate(fetch_these):
    if hasattr(fit, f):
        print(f)
        attr = getattr(fit,f)        
        if hasattr(attr, 'dtype'):
            # for numpy arrays# 
            print(attr.ndim)
            print(attr.shape)
            print(attr.dtype)
        if hasattr(attr, 'values'):
            # for pandas Series and Dataframes 
            #print('dtype', attr.dtype)
            print(attr.values.shape)
            print(attr.values.dtype)
        elif callable(attr):
            # for callable methods
            vals = eval('fit.' + attr.__name__ + '()')
            #pdb.set_trace()
            print(vals.values.shape)
            print(vals.values.dtype)
        else:
            # or die
            print(attr.__class__)
            print(dir(attr))
            msg = 'I dont know what to do with ' + attr
            raise TypeError(msg)
    else:
        msg = f + ' not found'
        print(dir(fit))
        raise ValueError(msg)
        
 

['HC0_se', 'HC1_se', 'HC2_se', 'HC3_se', '_HCCM', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_cache', '_data_attr', '_get_robustcov_results', '_is_nested', '_wexog_singular_values', 'aic', 'bic', 'bse', 'centered_tss', 'compare_f_test', 'compare_lm_test', 'compare_lr_test', 'condition_number', 'conf_int', 'conf_int_el', 'cov_HC0', 'cov_HC1', 'cov_HC2', 'cov_HC3', 'cov_kwds', 'cov_params', 'cov_type', 'df_model', 'df_resid', 'eigenvals', 'el_test', 'ess', 'f_pvalue', 'f_test', 'fittedvalues', 'fvalue', 'get_influence', 'get_prediction', 'get_robustcov_results', 'het_scale', 'initialize', 'k_constant', 'llf', 'load', 'model', 'mse_model', 'mse_resid', 'mse_total', 'nobs', 'normalized_cov_par

ValueError: coefs not found

In [ ]:
# Option 2: array of fit_coef_dt, where each one holds the
# parameter values for a single coef
fit_arry = np.ndarray(shape=(len(rhs_di.column_names)),
                   dtype=fit_coef_dt)

# since each item in the array has the same dtype we 
# can slice *across* the fields by name
print('value of coef for coefs: ' + ' '.join(rhs_di.column_names))
print(fit_arry['coef']) # value of coef for all the coefs
print()

# Disadvantage: we have access to a given parameter by index, to use names we have to lookup the index
for name,jdx in rhs_di.column_name_indexes.items():
    if name == 'x0':
        print('parameter: ', name, 'column index: ', jdx)
        print('ci: ', fit_arry[jdx]['ci']) # all the info about 